# H₂ Molecule Homework Assignment
### Quantum Software Development Journey: From Theory to Application with Classiq - Part 3

- Similarly to what we have done in class, in this exercise we will implement the VQE on H2 molecule.
- This time instead of using the built-in methods and functions (such as `Molecule` and `MoleculeProblem`) to difne and solve the problem, you will be provided with a two qubits Hamiltonian.

## Submission
- Submit the completed Jupyter notebook and report via GitHub. Ensure all files are correctly named and organized.
- Use the Typeform link provided in the submission folder to confirm your submission.

## Additional Resources
- [Classiq Documentation](https://docs.classiq.io/latest/)
- The notebook from live session #3

## Important Dates
- **Assignment Release:** 22.5.2024
- **Submission Deadline:** 3.6.2024 (7 A.M GMT+3)

---

Happy coding and good luck!

### Part 1

Given the following Hamiltonian:

$$
\hat{H} = -1.0523 \cdot (I \otimes I) + 0.3979 \cdot (I \otimes Z) - 0.3979 \cdot (Z \otimes I) - 0.0112 \cdot (Z \otimes Z) + 0.1809 \cdot (X \otimes X)
$$

Complete the following code

In [2]:
from typing import List
from classiq import *

HAMILTONIAN = QConstant("HAMILTONIAN", List[PauliTerm], 
                        [            
            PauliTerm([Pauli.I,Pauli.I], -1.0523),
            PauliTerm([Pauli.I,Pauli.Z], 0.3979),
            PauliTerm([Pauli.Z,Pauli.I], -0.3979),
            PauliTerm([Pauli.Z, Pauli.Z], -0.0112),
            PauliTerm([Pauli.X, Pauli.X], 0.1809)])

In [3]:
@qfunc
def main(q: Output[QArray], angles: CArray[CReal, 3]) -> None:
    allocate(2, q)
    U(angles[0], angles[1], angles[2], 0, q[0])
    U(angles[0], angles[1], angles[2], 0, q[1]) 


@cfunc
def cmain() -> None:
    res = vqe(
        HAMILTONIAN, # TODO: complete the missing argument
        False,
        [],
        optimizer=Optimizer.COBYLA,
        max_iteration=1000,
        tolerance=0.001,
        step_size=0,
        skip_compute_variance=False,
        alpha_cvar=1.0,
    )
    save({"result": res})

qmod = create_model(main, classical_execution_function=cmain)
qprog = synthesize(qmod)
# show(qprog)

In [7]:
execution = execute(qprog)
res = execution.result()
# execution.open_in_ide()
vqe_result = res[0].value
# res

In [8]:
print(f"Optimal energy: {vqe_result.energy}")
print(f"Optimal parameters: {vqe_result.optimal_parameters}")
print(f"Eigenstate: {vqe_result.eigenstate}")

Optimal energy: -1.0765193359374998
Optimal parameters: {'angles_0': 2.595605123495132, 'angles_1': 4.80469078930179, 'angles_2': -2.830882928496536}
Eigenstate: {'00': (0.06629126073623882+0j), '10': (0.240035804620894+0j), '01': (0.26052081250832915+0j), '11': (0.9328007222874563+0j)}


Does it similar to the `optimal energy` we calculated in class? \
Does it similar to the `total energy` we calculated in class?

No, it does not look equal, the value for optimal energy found in the class for H_{2} molecule as well as the 2 -qubit Hamiltonian was around -1.8681. The total energy for H_{2} Hamiltonian was around -1.13. close to the above result

### Part 2

**Now, we want to have a more interesting ansatz in our `main`.**  
Add **one** line of code to the `main` function you created in Part 1 that creates **entanglement** between the two qubits.  
Which gate should you use?

In [9]:
@qfunc
def main(q: Output[QArray[QBit]], angles: CArray[CReal, 3]) -> None:
   allocate(2, q)
   H(q[0])
   CX(q[0], q[1])
   U(angles[0], angles[1], angles[2], 0, q[0])
   U(angles[0], angles[1], angles[2], 0, q[1]) 


@cfunc
def cmain() -> None:
    res = vqe(
        HAMILTONIAN, # TODO: complete the missing argument
        False,
        [],
        optimizer=Optimizer.COBYLA,
        max_iteration=1000,
        tolerance=0.001,
        step_size=0,
        skip_compute_variance=False,
        alpha_cvar=1.0,
    )
    save({"result": res})


qmod = create_model(main, classical_execution_function=cmain)
qprog = synthesize(qmod)

In [10]:
execution = execute(qprog)
res = execution.result()
# execution.open_in_ide()
vqe_result = res[0].value

In [11]:
print(f"Optimal energy: {vqe_result.energy}")
print(f"Optimal parameters: {vqe_result.optimal_parameters}")
print(f"Eigenstate: {vqe_result.eigenstate}")

Optimal energy: -1.2425021484375
Optimal parameters: {'angles_0': 2.9032006536886388, 'angles_1': 1.2875932713219624, 'angles_2': -3.4443929142666185}
Eigenstate: {'01': (0.038273277230987154+0j), '10': (0.04419417382415922+0j), '00': (0.7115810960459813+0j), '11': (0.7001673907002525+0j)}


Does it similar to the `optimal energy` we calculated in class? \
Does it similar to the `total energy` we calculated in class? \
What can we learn about the provided  form this result Hamiltonian?

Energy values seem to get a bit closer to the energy values obtained for H_{2} ground state in the class, but I'm not sure why the Hamiltonian in this notebook even representing the Fermionic H_{2} Hamiltonian.

Choice of ansatz plays an important role in VQE estimates. 